In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns
sns.set()

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [3]:
# https://stackoverflow.com/questions/30557409/spotify-api-post-call-response-415
client_id = "9cd3dd2ea2cf492ca28ab0247a79d781"
client_secret = "11c972ad002843e9be5ecc31f022dd6e"
grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}
url = 'https://accounts.spotify.com/api/token'

response = requests.post(url, data=body_params, auth = (client_id, client_secret)) 
print(response.content)

b'{"access_token":"BQAyK0HLBesG-DG7MMd8uZfGV2VBXS3AKWfvpXuKxWHCuRuqq9h7bztsVGIZHhSphvYUpz3bq4UFnn3nCAc","token_type":"Bearer","expires_in":3600,"scope":""}'


In [4]:
# Using Spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

sp.audio_features(["0UaMYEvWZi0ZqiDOoHU3YI"])

[{'danceability': 0.904,
  'energy': 0.813,
  'key': 4,
  'loudness': -7.105,
  'mode': 0,
  'speechiness': 0.121,
  'acousticness': 0.0311,
  'instrumentalness': 0.00697,
  'liveness': 0.0471,
  'valence': 0.81,
  'tempo': 125.461,
  'type': 'audio_features',
  'id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
  'duration_ms': 226864,
  'time_signature': 4}]

In [5]:
def songs_in_playlist(df, pid):
    '''
    input: dataframe and playlist ID
    returns: list of all songs on playlist by track_uri 
    '''
    playlist = np.where(df.pid == pid)
    return list(df.iloc[playlist]["track_uri"])

def playlist_means(tracks):
    '''
    input: list of tracks on a playlist (can be generated from songs_in_playlist())
    returns: statistics for audio features of playlist  
    '''
    num = len(tracks)
    # We can only pass 100 tracks at a time
    if num <= 100:
        p = pd.DataFrame.from_records(sp.audio_features(tracks))
    else:
        start = 0
        end = 100
        p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
        for i in range(int(num / 100)):
            start = end
            diff = num - end
            if diff > 100:
                end += 100
            else:
                end += diff
            
            new_p = pd.DataFrame.from_records(sp.audio_features(tracks[start:end]))
            p = p.append(new_p) 
    return p

In [57]:
songs = pd.read_pickle("william/pickles/songs_30k_dropped.pkl")
songs.head()

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.03110,https://api.spotify.com/v1/audio-analysis/0UaM...,0.904,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.1210,125.461,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,1,0.02490,https://api.spotify.com/v1/audio-analysis/6I9V...,0.774,198800,0.838,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.1140,143.040,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,2,0.00238,https://api.spotify.com/v1/audio-analysis/0WqI...,0.664,235933,0.758,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.2100,99.259,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,3,0.20200,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.891,267267,0.714,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.1400,100.972,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,4,0.05610,https://api.spotify.com/v1/audio-analysis/1lzr...,0.853,227600,0.606,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.0713,94.759,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [52]:
songs2 = songs.copy()
songs2 = songs2.reset_index()
cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]

def scaler(cols, df_old, df_new):
    df_new = df_old.copy()
    x_scaler = MinMaxScaler().fit(df_old["tempo"].values.reshape(-1,1))
    x_scaled = x_scaler.transform(df_old["tempo"].values.reshape(-1,1))
    df_new["tempo"] = x_scaled

#     x_scaler.transform(df_old[col].values.reshape(-1,1))
    
#     for col in cols: 
#         x_scaler = MinMaxScaler().fit(df_old[col].values.reshape(-1,1))
#         x_scaled = MinMaxScaler().fit_transform(df_old[col].values.reshape(-1,1))
#         df_new[col] = x_scaled
        
#         x_scaler.transform(df_old[col].values.reshape(-1,1))
    return(df_new, x_scaler)


        
songs2, x_scaler = scaler(cols, songs, songs2)
songs2.head()
# songs2['index']

,index,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0,0.03110,https://api.spotify.com/v1/audio-analysis/0UaM...,0.904,226864,0.813,0UaMYEvWZi0ZqiDOoHU3YI,0.006970,4,0.0471,-7.105,0,0.1210,0.505967,4,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,audio_features,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0.810
1,1,0.02490,https://api.spotify.com/v1/audio-analysis/6I9V...,0.774,198800,0.838,6I9VzXrHxO9rA9A5euc8Ak,0.025000,5,0.2420,-3.914,0,0.1140,0.576860,4,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,audio_features,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0.924
2,2,0.00238,https://api.spotify.com/v1/audio-analysis/0WqI...,0.664,235933,0.758,0WqIKmW4BTrj3eJFmnCKMv,0.000000,2,0.0598,-6.583,0,0.2100,0.400298,4,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,audio_features,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0.701
3,3,0.20200,https://api.spotify.com/v1/audio-analysis/1AWQ...,0.891,267267,0.714,1AWQoqb9bSvzTjaLralEkT,0.000234,4,0.0521,-6.055,0,0.1400,0.407206,4,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,audio_features,spotify:track:1AWQoqb9bSvzTjaLralEkT,0.818
4,4,0.05610,https://api.spotify.com/v1/audio-analysis/1lzr...,0.853,227600,0.606,1lzr43nnXAijIGYnCT8M8H,0.000000,0,0.3130,-4.596,1,0.0713,0.382150,4,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,audio_features,spotify:track:1lzr43nnXAijIGYnCT8M8H,0.654


In [ ]:
song1 = songs2.iloc[np.where(songs.id == '1MYYt7h6amcrauCOoso3Gx')]
song1

In [ ]:
# def distance(idkey, songs):
#     song = pd.DataFrame(sp.audio_features(idkey))
#     song_scaled = song.copy()
# #     song = scaler(cols, song, song_scaled)
#     for col in ["tempo"]:
#         song_scaled[col] = x_scaler.transform(song.loc[:, col].values.reshape(-1,1))
#         print(col, song_scaled.loc[:, col])
        
#     song = song_scaled
#     dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
#                  + (np.subtract(songs.danceability, song.danceability))**2
#                  + (np.subtract(songs.energy, song.energy))**2
#                  + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
#                  + (np.subtract(songs.liveness, song.liveness))**2
#                  + (np.subtract(songs.speechiness, song.speechiness))**2
#                  + (np.subtract(songs.tempo, song.tempo))**2)
        
#     return dist


def playlist_generator(input_song_uri, n, songDF):
    def distance(idkey, songs):
        song = pd.DataFrame(sp.audio_features(idkey))
        song.loc[:,"tempo"] = x_scaler.transform(song.loc[:,"tempo"].values.reshape(-1,1))
        dist = np.sqrt((np.subtract(songs.acousticness, song.acousticness))**2
                     + (np.subtract(songs.danceability, song.danceability))**2
                     + (np.subtract(songs.energy, song.energy))**2
                     + (np.subtract(songs.instrumentalness, song.instrumentalness))**2
                     + (np.subtract(songs.liveness, song.liveness))**2
                     + (np.subtract(songs.speechiness, song.speechiness))**2
                     + (np.subtract(songs.tempo, song.tempo))**2)

        return dist
    
    topn_dist = distance(input_song_uri, songDF).sort_values()[0:n]
    print(topn_dist.index)
    topn = songs2.loc[topn_dist.index]
    return topn

def playlist_printer(topn_ids):
    for ref in topn_ids['id']:
        track = sp.track(ref)
        print(track['name'])
        print('by')
        for artist in track['artists']:
            print(artist['name'])
        print('-----------------------------------')
        

topn_ids = playlist_generator('5cYA45RVGI6F4f06gtWjsd', 25, songs2)
playlist_printer(topn_ids)

In [ ]:
top10_dist = distance('5IVuqXILoxVWvWEPm82Jxr', songs2).sort_values()[0:10]

In [ ]:
top10_dist.index

# k-means clustering

In [110]:
from sklearn.cluster import KMeans
import random

In [25]:
songs = pd.read_pickle("william/pickles/songs_30k_dropped.pkl")
songs_af = songs[["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]]

In [147]:
kmeans = KMeans(100).fit(songs_af)

In [1]:
songs_w_cluster = songs_af.copy()
songs_w_cluster["cluster"] = kmeans.predict(songs_af)
songs_w_cluster.head()

# maxc = 0
# for i in range(100):
#     count = len(songs_w_cluster[songs_w_cluster.cluster == i])
#     print(i, count)
#     if count > maxc:
#         maxc = count
        
# print(maxc)

NameError: name 'songs_af' is not defined

In [143]:
kmeans.cluster_centers_

In [167]:
def centers_to_songs(cluster_centers, songs):
    '''
    convert the k-means cluster centers into the nearest songs
    
    cluster_centers: cluster centers from kmeans model
    songs: dataframe of songs
    
    return: c2s - dictionary mapping cluster label to the nearest song index in the dataframe
    '''
    c2s = {}
    for i in range(cluster_centers.shape[0]):
        dist = np.sqrt((np.subtract(songs.acousticness, cluster_centers[i, 0]))**2
                     + (np.subtract(songs.danceability, cluster_centers[i, 1]))**2
                     + (np.subtract(songs.energy, cluster_centers[i, 2]))**2
                     + (np.subtract(songs.instrumentalness, cluster_centers[i, 3]))**2
                     + (np.subtract(songs.liveness, cluster_centers[i, 4]))**2
                     + (np.subtract(songs.speechiness, cluster_centers[i, 5]))**2
                     + (np.subtract(songs.tempo, cluster_centers[i, 6]))**2)
        
        top_song_i = dist.sort_values()[0:2].index[0]
        c2s[i] = top_song_i

    return c2s

def song_printer(ids, df):
    '''
    print out the playlist from song indexes and return the audio features of the playlist
    
    ids: list of song indexes in df
    df: dataframe containing songs
    
    return: playlist_features - dataframe of the audio features of the playlist
    '''
    cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]
    playlist_features = pd.DataFrame(columns = cols)
    
    for i in ids:
        track_uri = df.loc[i].id
        track = sp.track(track_uri)
        print(track['name'])
        print('by')
        for artist in track['artists']:
            print(artist['name'])
        print('-----------------------------------')
        
        playlist_features = playlist_features.append(pd.DataFrame(sp.audio_features(track_uri))[cols])
        
    return playlist_features

In [187]:
ids = centers_to_songs(kmeans.cluster_centers_, songs)
ids_lst = [x[1] for x in ids.items()]
song_printer(ids_lst, songs)

Un Poquito Más
by
Pedro Capó
-----------------------------------
Harrison Ford
by
Someone Still Loves You Boris Yeltsin
-----------------------------------
Real Shit
by
90 Pounds of Pete
-----------------------------------
Waiting On the River to Rise
by
Kingsley Flood
-----------------------------------
Copper Wire
by
The Range
-----------------------------------
That's Life (SMASH Cast Version) [feat. Megan Hilty & Katharine McPhee]
by
SMASH Cast
-----------------------------------
Level Up
by
Vienna Teng
-----------------------------------
Deep In The Heart
by
U2
-----------------------------------
Through the Chain Link
by
Stick To Your Guns
-----------------------------------
Zip-A-Dee-Doo-Dah
by
Ric Ocasek
-----------------------------------
Humpin' Around - Prelude
by
Bobby Brown
-----------------------------------
Deep Blood Red
by
Mali Music
-----------------------------------
Six More Miles
by
Mac Wiseman
-----------------------------------
Nuages
by
Django
------------------

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo
0,0.1530,0.520,0.6950,0,0.2680,0.1110,194.180
0,0.3690,0.626,0.5890,0.0145,0.1610,0.0283,113.078
0,0.2020,0.590,0.6780,0.142,0.2110,0.0300,131.979
0,0.2980,0.608,0.5290,1.59e-06,0.2150,0.0376,88.477
0,0.1370,0.517,0.6640,0.0688,0.1620,0.0435,159.949
0,0.5250,0.496,0.4360,0,0.1540,0.0562,76.563
0,0.2270,0.513,0.6810,0.0621,0.1110,0.0417,170.038
0,0.3860,0.494,0.5630,0.0781,0.2260,0.0298,143.744
0,0.2980,0.552,0.6170,4.67e-05,0.2000,0.0404,102.512
0,0.1160,0.601,0.7400,0.0412,0.1370,0.0302,126.370


In [169]:
# given a seed song, compute the audio features, find which cluster it is predicted to, return that cluster
def kmeans_song_to_playlist(song_id, kmeans, n, clustered_df):
    '''
    Given a seed song (track_uri), return a list of n song indexes in the dataframe which will be the playlist
    
    song_id: seed song uri
    kmeans: k-means clustering model
    n: number of songs in playlist
    clustered_df: songs dataframe with cluster labels predicted from kmeans model
    
    return: playlist_ids - list of n song indexes in the dataframe which is the playlist
    '''
    cols = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "speechiness", "tempo"]
    song_af = pd.DataFrame(sp.audio_features(song_id))
    pred_cluster = kmeans.predict(song_af[cols])[0]
    pred_cluster_songs = clustered_df[clustered_df == pred_cluster]
    
    playlist_ids = random.sample(list(pred_cluster_songs.index), n)

    return playlist_ids

In [172]:
# Senorita = "0TK2YIli7K1leLovkQiNik"
# Yo-Yo Ma Cello Suite No. 1 in G Major = "61dYvvfIRtIDFuqZypPAta"
ids = kmeans_song_to_playlist("61dYvvfIRtIDFuqZypPAta", kmeans, 10, songs_w_cluster)
playlist_features = song_printer(ids, songs)
# playlist_features.describe()

Louie Louie
by
The Swamp Rats
-----------------------------------
Demons - Original Mix
by
Paris Blohm
Charles
-----------------------------------
Resolution
by
Matt Corby
-----------------------------------
Sea Of Voices - Galimatias Remix
by
Porter Robinson
Galimatias
-----------------------------------
The Witch (Sensualise Remix)
by
John Askew
-----------------------------------
Bring On Summer
by
Chase Rice
-----------------------------------
Que Vuelvas
by
Shakira
-----------------------------------
Te Regalo La Lluvia
by
Ana Bárbara
-----------------------------------
Unforgettable
by
Drake
Jeezy
-----------------------------------
L'cha Dodi – Facing Shabbat
by
Nava Tehila
-----------------------------------
